### Thiết lập và truy xuất siêu tham số (hyperparameters) trong scikit-learn
#### Siêu tham số là gì? 
- Siêu tham số (hyperparameters) là những tham số điều khiển quá trình học của mô hình dự đoán.
- Chúng khác nhau giữa các loại mô hình, và phụ thuộc vào từng bộ dữ liệu, do đó luôn cần được tối ưu hóa.  

🔸 Lưu ý: Đừng nhầm lẫn siêu tham số với tham số đã được học (fitted parameters) sau khi huấn luyện, ví dụ model.coef_. Những tham số đã học này thường có hậu tố _ trong tên.



In [2]:
import pandas as pd

adult_census= pd.read_csv('../datasets/adult-census.csv')

target_name = 'class'
numerical_columns = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']

target_name = 'class'
data = adult_census[numerical_columns]
target = adult_census[target_name]

In [3]:
data

,age,capital-gain,capital-loss,hours-per-week
0,25,0,0,40
1,38,0,0,50
2,28,0,0,40
3,44,7688,0,40
4,18,0,0,30
...,...,...,...,...
48837,27,0,0,38
48838,40,0,0,40
48839,58,0,0,40
48840,22,0,0,20


In [7]:
for parameter in model.get_params():
    print(parameter)

memory
steps
verbose
standardscaler
logisticregression
standardscaler__copy
standardscaler__with_mean
standardscaler__with_std
logisticregression__C
logisticregression__class_weight
logisticregression__dual
logisticregression__fit_intercept
logisticregression__intercept_scaling
logisticregression__l1_ratio
logisticregression__max_iter
logisticregression__multi_class
logisticregression__n_jobs
logisticregression__penalty
logisticregression__random_state
logisticregression__solver
logisticregression__tol
logisticregression__verbose
logisticregression__warm_start


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = make_pipeline(
    StandardScaler(), 
    LogisticRegression()
)

In [5]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target)
scores = cv_results['test_score']

print(
    "Accuracy score via cross-validaton: "
    f"{scores.mean():.3f} +/- "
    f"{scores.std():.3f}"
)

Accuracy score via cross-validaton: 0.800 +/- 0.003


In [9]:
model.set_params(logisticregression__C=1e-3)
cv_results = cross_validate(
    model,
    data,
    target
)
scores = cv_results['test_score']
print(
    'Accuracy score via cross-validation: '
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

Accuracy score via cross-validation: 0.787 +/- 0.002


Có thể thay đổi hệ thống giá trị của siêu tham số `C` để xem liệu có giá trị nào tối ưu không 

In [12]:
for C in [1e-3, 1e-2, 1e-1, 1, 10, 100]:
    model.set_params(logisticregression__C=C)
    cv_results = cross_validate(
        model,
        data,
        target
    )

    scores = cv_results['test_score']
    print(
    f'Accuracy score via cross-validation with C={C}: '
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

Accuracy score via cross-validation with C=0.001: 0.787 +/- 0.002
Accuracy score via cross-validation with C=0.01: 0.799 +/- 0.003
Accuracy score via cross-validation with C=0.1: 0.800 +/- 0.003
Accuracy score via cross-validation with C=1: 0.800 +/- 0.003
Accuracy score via cross-validation with C=10: 0.800 +/- 0.003
Accuracy score via cross-validation with C=100: 0.800 +/- 0.003


- Chúng ta có thể thấy rằng miễn là giá trị `C` đủ cao, mô hình dường như hoạt động tốt.

- Điều chúng ta làm ở đây khá thủ công: nó bao gồm việc quét các giá trị của `C` và chọn giá trị tốt nhất một cách thủ công. Trong bài học tiếp theo, chúng ta sẽ học cách làm điều này tự động.

!!!! Cảnh báo:

- Khi chúng ta đánh giá một nhóm các mô hình trên dữ liệu kiểm tra và chọn mô hình hoạt động tốt nhất, chúng ta không thể tin tưởng vào độ chính xác dự đoán tương ứng, và cần phải áp dụng mô hình đã chọn cho dữ liệu mới. Thực tế, dữ liệu kiểm tra đã được sử dụng để chọn mô hình, vì vậy nó không còn độc lập với mô hình này nữa.

- Trong tập tin này, chúng ta đã thấy: Cách sử dụng `get_params` và `set_params` để lấy và thiết lập các siêu tham số của một mô hình.